In [153]:
import geopandas as gp
import pandas as pd
import os

In [154]:
from geopy.distance import distance

In [185]:
# # Derive block group geometries from blocks and save result for reuse later.

# # https://www2.census.gov/geo/pdfs/reference/GARM/Ch11GARM.pdf
# repo_path = '/Users/rsharp/repos/housing_equity'
# shapefile = 'shapefiles/WACensusBlocks2010/tl_2019_53_tabblock10.shp'

# gdf = gp.read_file(os.path.join(repo_path, shapefile))
# gdf['block_group'] = gdf['BLOCKCE10'].str.get(0)
# gdf['block_group_geoid'] = gdf['STATEFP10'].str.cat(gdf['COUNTYFP10'].str.cat(gdf['TRACTCE10'].str.cat(gdf['block_group'])))

# # gdf[['block_group_geoid', 'geometry']].groupby('block_group_geoid').count()

# bg_shapefile = 'shapefiles/wa_king_census_block_groups.geojson'
# bg_gdf = gdf[(gdf['STATEFP10'] == state) & (gdf['COUNTYFP10'] == county)] \
#             [['block_group_geoid', 'geometry']].dissolve(by='block_group_geoid')
# bg_gdf.to_file(os.path.join(repo_path, bg_shapefile), driver='GeoJSON')

In [186]:
repo_path = '/Users/rsharp/repos/housing_equity'
shapefile = 'shapefiles/wa_king_census_block_groups.geojson'

bg_gdf = gp.read_file(os.path.join(repo_path, shapefile))

In [187]:
# State and County codes: https://www.census.gov/prod/techdoc/cbp/cbp95/st-cnty.pdf
state = '53'
county = '033'

king_gdf = bg_gdf

king_gdf['centroid'] = king_gdf['geometry'].centroid

In [207]:
n = king_gdf.shape[0]
r = (n*n - n)/2.0
r0 = (1000*1000 - 1000)/2.0
n, r, r/r0*4

(1422, 1010331.0, 8.09073873873874)

In [208]:
%%time

# Note that essentially all of the time is consumed by the distance calculation. This means that it should scale
# linearly with the number of pairs of centroids. It took about 4 minutes to compute for 1,000 block groups, which
# works out to (1000*1000 - 1000)/2 - 1000 = 499,500 pairs. The full King county dataset should take about 8 minutes.

s = king_gdf[['block_group_geoid', 'centroid']].copy()
s['temp_key'] = 1
xs = s.merge(s, on='temp_key', suffixes=('_a', '_b'))
xs = xs[xs['block_group_geoid_a'] > xs['block_group_geoid_b']]
xs['distance'] = xs.apply(lambda row: distance((row.centroid_a.y, row.centroid_a.x), (row.centroid_b.y, row.centroid_b.x)), axis=1)
xs[['block_group_geoid_a', 'block_group_geoid_b', 'distance']]

CPU times: user 6min 31s, sys: 33.3 s, total: 7min 5s
Wall time: 7min 11s


,block_group_geoid_a,block_group_geoid_b,distance
1422,530330001002,530330001001,0.9394085593676534 km
2844,530330001003,530330001001,0.7819418715368167 km
2845,530330001003,530330001002,0.7578799583027931 km
4266,530330001004,530330001001,1.1971442067327784 km
4267,530330001004,530330001002,0.32703321514162215 km
...,...,...,...
2022078,530339901000,530330327043,47.80899920262759 km
2022079,530339901000,530330327044,48.045810728719886 km
2022080,530339901000,530330328001,49.872263328947724 km
2022081,530339901000,530330328002,57.611324793142856 km


In [213]:
csv_filename = 'shapefiles/wa_king_census_block_groups_distances.csv'

full_filename = os.path.join(repo_path, csv_filename)
csv_data = xs[['block_group_geoid_a', 'block_group_geoid_b', 'distance']]
csv_data.to_csv(full_filename, header=True, index=False)
print('Wrote {} records to {}'.format(csv_data.shape[0], full_filename))

Wrote 1010331 records to /Users/rsharp/repos/housing_equity/shapefiles/wa_king_census_block_groups_distances.csv
